# data preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pprint
import operator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import preprocessing

In [3]:
data = pd.read_excel('/content/drive/My Drive/Siscer/stroke.xlsx')
print(data.head())

   gender   age  hypertension  heart_disease  ever_married  work_type  \
0       1  67.0             0              1             1          0   
1       1  80.0             0              1             1          0   
2       0  49.0             0              0             1          0   
3       0  79.0             1              0             1          1   
4       1  81.0             0              0             1          0   

   Residence_type  avg_glucose_level   bmi  smoking_status  stroke  
0               1             228.69  36.6               2       1  
1               0             105.92  32.5               1       1  
2               1             171.23  34.4               3       1  
3               0             174.12  24.0               1       1  
4               1             186.21  29.0               2       1  


**EDA**

In [4]:
# Display basic information about the DataFrame
print(data.info())

print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   int64  
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   int64  
 5   work_type          4981 non-null   int64  
 6   Residence_type     4981 non-null   int64  
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   int64  
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(8)
memory usage: 428.2 KB
None
            gender          age  hypertension  heart_disease  ever_married  \
count  4981.000000  4981.000000   4981.000000    4981.000000   4981.000000   
mean      0.416382    43.419859      0.096165       0.05

In [5]:
X = data.iloc[:,0:10]
y = data.iloc[:,10]

#split dataset 70:30
xTrain, xTest, yTrain, yTest = train_test_split(X,y,test_size=0.30)
xTrain = xTrain.to_numpy()
xTest = xTest.to_numpy()
yTrain = yTrain.to_numpy()
yTest = yTest.to_numpy()

**FuzzyKNN**

In [6]:
class FuzzyKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3, plot=False):
        self.k = k
        self.plot = plot


    def fit(self, X, y=None):
        self._check_params(X,y)
        self.X = X
        self.y = y

        self.xdim = len(self.X[0])
        self.n = len(y)

        classes = list(set(y))
        classes.sort()
        self.classes = classes

        self.df = pd.DataFrame(self.X)
        self.df['y'] = self.y

        self.memberships = self._compute_memberships()

        self.df['membership'] = self.memberships

        self.fitted_ = True
        return self


    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('predict() called before fit()')
        else:
            m = 2
            y_pred = []

            for x in X:
                neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)

                votes = {}
                for c in self.classes:
                    den = 0
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                        den += 1 / (dist ** (2 / (m-1)))

                    neighbors_votes = []
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                        num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))

                        vote = num/den
                        neighbors_votes.append(vote)
                    votes[c] = np.sum(neighbors_votes)

                pred = max(votes.items(), key=operator.itemgetter(1))[0]
                y_pred.append((pred, votes))

            return y_pred


    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('score() called before fit()')
        else:
            predictions = self.predict(X)
            y_pred = [t[0] for t in predictions]
            confidences = [t[1] for t in predictions]

            return accuracy_score(y_pred=y_pred, y_true=y)


    def _find_k_nearest_neighbors(self, df, x):
        X = df.iloc[:,0:self.xdim].values

        df['distances'] = [np.linalg.norm(X[i] - x) for i in range(self.n)]

        df.sort_values(by='distances', ascending=True, inplace=True)
        neighbors = df.iloc[0:self.k]

        return neighbors


    def _get_counts(self, neighbors):
        groups = neighbors.groupby('y')
        counts = {group[1]['y'].iloc[0]:group[1].count()[0] for group in groups}

        return counts


    def _compute_memberships(self):
        memberships = []
        for i in range(self.n):
            x = self.X[i]
            y = self.y[i]

            neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
            counts = self._get_counts(neighbors)

            membership = dict()
            for c in self.classes:
                try:
                    uci = 0.49 * (counts[c] / self.k)
                    if c == y:
                        uci += 0.51
                    membership[c] = uci
                except:
                    membership[c] = 0

            memberships.append(membership)
        return memberships


    def _check_params(self, X, y):
        if type(self.k) != int:
            raise Exception('"k" should have type int')
        elif self.k >= len(y):
            raise Exception('"k" should be less than no of feature sets')
        elif self.k % 2 == 0:
            raise Exception('"k" should be odd')

        if type(self.plot) != bool:
            raise Exception('"plot" should have type bool')

In [7]:
#Fuzzy KNN
fkModel = FuzzyKNN()
fkModel.fit(xTrain, yTrain)
y_predFK=fkModel.predict(xTest)

#Hilangkan indeks pertama
new_y_predFK = []
for item in y_predFK :
    new_y_predFK.append(item[1])
y_predFK = new_y_predFK

#ubah ke ndarray
new_y_predFK = []
for item in y_predFK :
    new_y_predFK.append([item[0],item[1]])
fk_proba = np.array(new_y_predFK)

In [8]:
#akurasi Fuzzy KNN
y_pred_fk = np.argmax(fk_proba, axis=1)
accuracy_score(y_pred_fk,yTest)

0.9391304347826087

Optimasi GA

In [9]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.4 MB/s eta 0:00:00


In [10]:
from deap import base, creator, tools, algorithms
import random

In [ ]:
data.reset_index(drop=True, inplace=True)

def evaluate(individual):
    # Example fitness function: sum of values from selected columns
    # Assuming your CSV has columns named 'A', 'B', 'C', etc.
    # Modify this function based on your problem
    selected_columns = ['gender', 'age', 'hypertension','heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level','bmi','smoking_status','stroke']
    valid_indices = [index for index in individual if index < len(data)]

    # Calculate the sum of selected columns for the selected rows
    subset_df = data.loc[valid_indices, selected_columns]
    total_sum = subset_df.sum().sum()

    return total_sum,  # Comma makes it a tuple for DEAP

In [ ]:
# Set up the genetic algorithm
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize the fitness function
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(len(data)), len(data))  # Randomly sample indices
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Parameters for the genetic algorithm
population_size = 4981
num_generations = 50
crossover_probability = 0.7
mutation_probability = 0.2

# Create the initial population
population = toolbox.population(n=population_size)

# Run the genetic algorithm
algorithms.eaSimple(population, toolbox, cxpb=crossover_probability, mutpb=mutation_probability, ngen=num_generations, verbose=True)

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
print("Best individual:", best_individual)
print("Fitness:", evaluate(best_individual))


gen	nevals
0  	4981  
1  	3777  
2  	3758  
3  	3762  
4  	3821  
5  	3774  
6  	3824  
7  	3755  
8  	3813  
9  	3828  
10 	3825  
11 	3785  
12 	3775  
13 	3795  
14 	3680  
15 	3751  
16 	3739  
17 	3745  
18 	3798  
19 	3825  
20 	3800  
21 	3787  
22 	3768  
23 	3841  
24 	3813  
25 	3807  
26 	3759  
27 	3840  
28 	3784  
29 	3760  
30 	3763  
31 	3769  
32 	3827  
33 	3782  
34 	3767  
35 	3744  
36 	3790  
37 	3826  
38 	3692  
39 	3829  
40 	3791  
41 	3827  
42 	3740  
43 	3767  
44 	3753  
45 	3737  
46 	3829  
47 	3749  
48 	3733  
49 	3785  
50 	3795  
Best individual: [3358, 4463, 3592, 3425, 2981, 4513, 597, 2175, 2354, 913, 4308, 4323, 1270, 3156, 1912, 1606, 2495, 1265, 1399, 4688, 4203, 4096, 3463, 1725, 2329, 617, 4303, 3723, 4408, 1836, 861, 1724, 1829, 4829, 1749, 3082, 3019, 661, 4389, 176, 1280, 2807, 1799, 3355, 3595, 2303, 1028, 4640, 4621, 974, 2198, 2521, 3758, 738, 749, 837, 2395, 1339, 1712, 1227, 2793, 1247, 908, 1176, 1783, 4921, 556, 2244, 1004, 1577, 36

In [12]:
  # k should be an odd number between 1 and 19

# The rest of the genetic algorithm setup remains the same


In [ ]:
#kodedian
# Define evaluate function for GA
import random
def evaluate(individual):
    k = int(individual[0])
    features = [bool(f) for f in individual[1:]]

    # Select features based on individual
    X_train_selected = xTrain[:, features]
    X_test_selected = xTest[:, features]

    # Create FuzzyKNN model with current k and selected features
    model = FuzzyKNN(k=k)
    model.fit(X_train_selected, yTrain)
    predictions = model.predict(X_test_selected)
    y_pred = [t[0] for t in predictions]

    # Calculate accuracy
    accuracy = accuracy_score(yTest, y_pred)
    return accuracy,

# GA setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def attr_k_odd():
    return random.choice(range(1, 20, 2))

toolbox = base.Toolbox()
toolbox.register("attr_k", attr_k_odd)  # k should be an odd number between 1 and 19
toolbox.register("attr_bool", random.randint, 0, 1)  # Feature selection (0 or 1)

# Define individual and population
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_k,) + (toolbox.attr_bool,) * X.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# GA parameters
population_size = 100
num_generations = 50
crossover_probability = 0.7
mutation_probability = 0.2

# Create initial population
population = toolbox.population(n=population_size)

# Run GA
algorithms.eaSimple(population, toolbox, cxpb=crossover_probability, mutpb=mutation_probability,
                    ngen=num_generations, verbose=True)

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
print("Best individual:", best_individual)
print("Fitness:", evaluate(best_individual))

# Train final model with optimized parameters
best_k = int(best_individual[0])
best_features = [bool(f) for f in best_individual[1:]]
xTrain_selected = xTrain[:, best_features]
xTest_selected = xTest[:, best_features]

final_model = FuzzyKNN(k=best_k)
final_model.fit(xTrain_selected, yTrain)
final_predictions = final_model.predict(xTest_selected)
final_y_pred = [t[0] for t in final_predictions]

# Final accuracy
final_accuracy = accuracy_score(yTest, final_y_pred)
print("Final Fuzzy KNN Accuracy with GA:", final_accuracy)

Streaming output truncated to the last 5000 lines.
<ipython-input-6-db25e439682b>:45: RuntimeWarning: divide by zero encountered in scalar divide
  den += 1 / (dist ** (2 / (m-1)))
<ipython-input-6-db25e439682b>:50: RuntimeWarning: divide by zero encountered in scalar divide
  num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
<ipython-input-6-db25e439682b>:52: RuntimeWarning: invalid value encountered in scalar divide
  vote = num/den
<ipython-input-6-db25e439682b>:50: RuntimeWarning: invalid value encountered in scalar divide
  num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
<ipython-input-6-db25e439682b>:45: RuntimeWarning: divide by zero encountered in scalar divide
  den += 1 / (dist ** (2 / (m-1)))
<ipython-input-6-db25e439682b>:50: RuntimeWarning: divide by zero encountered in scalar divide
  num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
<ipython-input-6-db25e439682b>:52: RuntimeWarning: invalid value encountered in scalar divide
  